In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 14:19:38--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-27 14:19:38 (11.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
import minsearch

In [5]:
import json

In [6]:
# Load json file

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
# Organize json into a list of dictionaries

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        

In [8]:
# keyword_fields are used for filtering (i.e. only answers about Data Engineering Course)
# text_fields contain relevant information
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
# Index list of dictionaries documents
index.fit(documents)

In [15]:
# Perform a search
q = 'the course has already started, can I still enroll?'

# If one text_fields is more important than others, we can boost it (term from elasticsearch)
# In this case, question field is 3x more important, section is half as important
# We choose this because we are attempting to match our query to the most similar question already asked in FAQ documents
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    filter_dict = {'course': 'data-engineering-zoomcamp'}, # optional if we want to filter
    num_results = 5 # how many to return
)

In [27]:
# Display search results

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI

In [11]:
# note: must expose OpenAI API key in environmental variable, see week 1 notes

client = OpenAI()

In [19]:
# General response from gpt based on the question (q) from earlier

response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages =[{"role": "user", "content": q}]
)

In [22]:
response.choices[0].message.content

"It depends on the policies of the institution or organization offering the course. Some may allow late enrollment with certain conditions, such as catching up on missed work or material. It's best to contact the course administrator or instructor to inquire about late enrollment options."

In [23]:
# Engineer a prompt template (with role) to give to LLM
# This is important for GPT3.5 - 4
# This is more of an art then a science (prompt engineering...)
# Later will do prompt analysis with metrics to help guide

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}
""".strip()
# remove line breaks with strip

In [30]:
# Format the results from our index search

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [34]:
# Generate prompt using the template and the context

prompt = prompt_template.format(question=q, context=context).strip()

# You'll see that we have a defined role, question, and context within prompt now
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [37]:
# Ask GPT with context which is the previously searched info from the FAQ database
# NOTE: attempted with gpt-3.5-turbo and received NONE back

response = client.chat.completions.create(
    model = 'gpt-4o',
    messages =[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [38]:
# Again process is:
# 1. Ask Question
# 2. Search database for relevant context
# 3. Provide question and context to LLM -> can try various services (see Databricks or AWS as an option)
# 4. Receive natural language response back

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    results = index.search(
        query = query,
        boost_dict = boost,
        filter_dict = {'course': 'data-engineering-zoomcamp'}, # optional if we want to filter
        num_results = 5 # how many to return
    )

    return results

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return(prompt)

In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages =[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)

    return answer

In [21]:
query = "how do I run kafka?"
rag(query)

'To run Kafka in Java, you can follow these steps:\n\nIn the project directory, run:\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nEnsure to replace `<jar_name>` with the actual name of your jar file.'

In [22]:
# Elasticsearch saves the data on disk so you don't need to re-execute training
from elasticsearch import Elasticsearch

In [27]:
es_client = Elasticsearch('http://localhost:9200')

In [30]:
es_client.info()

ObjectApiResponse({'name': '232724a81a1f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'fNduRiMvTG2ptQOTiD0n7w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [31]:
# Setup Elasticsearch

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/bn12Eac4RmS6uavCQ1erLw] already exists')

In [32]:
from tqdm.auto import tqdm

/Users/davewilson/Documents/llm-zoomcamp/llm_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

Python(37971) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:31<00:00, 29.92it/s]


In [34]:
query = "I just discovered the course. Can I still join it?"

In [40]:
def elastic_search(query):
    
    search_query = {
        "size": 5, # return 5 answers
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # Increasing importance for question field like before
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name,body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [42]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)

    return answer

In [43]:
rag(query)

"Yes, you can still join the course. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."